In [1]:
import warnings
warnings.filterwarnings("ignore")

import pickle
import pyscf
import numpy as np
from pyscf import gto, dft
from sigma.sigma import SIGMA
from sigma.usigma import USIGMA

In [2]:
print('pyscf version:', pyscf.__version__)

pyscf version: 2.5.0


In [3]:
mol = gto.Mole()
mol.verbose = 0
mol.atom = [
    [6 , (0. , 0. ,-0.646514)],
    [8 , (0. , 0. , 0.484886)]]
mol.basis = 'ccpvtz'
mol.build()

mf = dft.RKS(mol, xc='pbe').density_fit().run()

sigma = SIGMA(mf)
sigma.kernel(pkl="co.pkl")
print(f'RPA:   E_corr={sigma.e_corr_rpa:.10f}  E_tot={sigma.e_tot_rpa:.10f}')
print(f'SIGMA: E_corr={sigma.e_corr:.10f}  E_tot={sigma.e_tot:.10f}')

RPA:   E_corr=-0.5869782353  E_tot=-113.3481242528
SIGMA: E_corr=-0.6379984552  E_tot=-113.3991444726


In [4]:
def read_pkl(pkl):
    with open(pkl, "rb") as fileObj:
        e_hf = pickle.load(fileObj)
        wts = pickle.load(fileObj)
        allsigmas = pickle.load(fileObj)
    return e_hf, wts, allsigmas

In [5]:
def eval_e_corr_rpa(wts, allsigmas):
    e_corr_rpa = 0.0
    for w in range(wts.shape[0]):
        ec_w_rpa = 0.0
        for sigma in allsigmas[w, :]:
            if sigma > 0.0:
                ec_w_rpa += np.log(1.0 + sigma) - sigma
            else:
                assert abs(sigma) < 1.0e-14
        e_corr_rpa += 1.0 / (2.0 * np.pi) * ec_w_rpa * wts[w]
    return e_corr_rpa

In [6]:
e_hf, wts, allsigmas = read_pkl("co.pkl")
e_corr_rpa = eval_e_corr_rpa(wts, allsigmas)
print(f'RPA:   E_corr={e_corr_rpa:.10f}  E_tot={e_hf+e_corr_rpa:.10f}')

RPA:   E_corr=-0.5869782353  E_tot=-113.3481242528


In [7]:
mol = gto.Mole()
mol.verbose = 0
mol.atom = [
    [7 , (0. , 0. , 0.129649)],
    [1 , (0. , 0. ,-0.907543)]]
mol.basis = {'N': 'augccpwcvtz', 'H': 'augccpvtz'}
mol.spin = 2
mol.build()

mf = dft.UKS(mol, xc='pbe').density_fit().run()

sigma = USIGMA(mf)
sigma.kernel(pkl="nh.pkl")
print(f'RPA:   E_corr={sigma.e_corr_rpa:.10f}  E_tot={sigma.e_tot_rpa:.10f}')
print(f'SIGMA: E_corr={sigma.e_corr:.10f}  E_tot={sigma.e_tot:.10f}')

RPA:   E_corr=-0.3609952264  E_tot=-55.3349335473
SIGMA: E_corr=-0.3123859626  E_tot=-55.2863242835


In [8]:
e_hf, wts, allsigmas = read_pkl("nh.pkl")
e_corr_rpa = eval_e_corr_rpa(wts, allsigmas)
print(f'RPA:   E_corr={e_corr_rpa:.10f}  E_tot={e_hf+e_corr_rpa:.10f}')

RPA:   E_corr=-0.3609952264  E_tot=-55.3349335473
